<a href="https://colab.research.google.com/github/cezarviana/fake-news-no-more/blob/main/test3_fake_news_no_more_(terra_plana).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%pip -q install google-genai


# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk


from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Priorize fontes jornalísticas reconhecidas e com boa reputação.
        Selecione no máximo 5 lançamentos que demonstrem relevância (baseada na cobertura e qualidade da fonte) e sejam os mais atuais possíveis, desde que sejam fontes jornalísticas reconhecidas e com boa reputação.
        Para cada lançamento relevante, forneça o título, um breve resumo e o **link direto para a notícia**.
        Se o tópico não possuir 5 notícias a seu respeito, apresente somente as encontradas. Sem adicionar outras notícias com termos parecidos e que não tenham relação direta com o tópico.
        Se o tópico gerar pouca cobertura noticiosa ou reações limitadas, sinalize essa baixa relevância como um possível indicativo para considerar outros tópicos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca de notícias no Google Search sobre o tópico",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


################################################
# --- Agente 2: Verificador de Fontes --- #
################################################
def agente_verificador_fontes(topico, lancamentos_buscados):
    verificador_fontes = Agent(
        name="agente_verificador_fontes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Fontes #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de fontes, especialista em fact-checking.
        Para cada fonte principal identificada:
        - Determine se o site ou canal é conhecido e geralmente considerado confiável.
        - Localize e examine a seção 'Sobre nós' (ou equivalente) para entender a missão, equipe e possíveis vieses do site.
        - Verifique se outras fontes confiáveis corroboram as informações apresentadas pela fonte principal. Liste as fontes que confirmam os achados.
        """,
        description="Agente que verifica as fontes levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fontes = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_fontes = call_agent(verificador_fontes, entrada_do_agente_verificador_fontes)
    return verificacao_fontes


################################################
# --- Agente 3: Verificador de Conteúdo --- #
################################################
def agente_verificador_conteudo(topico, lancamentos_buscados):
    verificador_conteudo = Agent(
        name="agente_verificador_conteudo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Conteúdo #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de conteúdo, especialista em fact-checking.
        Examine o conteúdo das notícias e informações relacionadas ao tópico.
        Linguagem e Estilo:
        - Avalie se há uso de sofismos ou outras técnicas de persuasão manipuladoras.
        - Identifique erros de ortografia e gramática que possam indicar falta de profissionalismo ou revisão.
        - Distinga claramente entre fatos apresentados e opiniões, verificando se as opiniões são devidamente atribuídas.
        Contexto e Evidências:
        - Verifique se a data da informação é relevante para o contexto atual.
        - Se houver imagens, utilize a busca reversa (Google Imagens) para verificar sua autenticidade e se foram usadas em outros contextos enganosos.
        Coerência:
        - Avalie a lógica interna do conteúdo e sua coerência com informações de outras fontes.
        """,
        description="Agente que verifica o conteúdo das notícias levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_conteudo = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_conteudo = call_agent(verificador_conteudo, entrada_do_agente_verificador_conteudo)
    return verificacao_conteudo


##########################################
# --- Agente 4: Agência de Fact-Checking --- #
##########################################
def agente_verificador_fatos(topico, data_de_hoje):
    verificador_fatos = Agent(
        name="agente_verificador_fatos",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua tarefa é verificar se o tópico/afirmação já foi alvo de checagem por agências de fact-checking confiáveis, através da  busca google (google_search), priorizando as verificações mais recentes sobre o tópico.
        As agências de fact-checking a serem consultadas de acordo com o tópico serão as indicadas abaixo:
        - No Brasil:
          - Aos Fatos;
          - Lupa;
          - UOL Confere;
          - Estadão Verifica;
          - Fato ou Fake (G1);
          - Boatos.org;
          - Agência Pública - Truco no Congresso;
          - Comprova;
          - E-farsas;
          - É isso Mesmo? (O Globo);
          - Portal EBC - Checagem;
        - Internacionais com atuação ou relevância no Brasil:
          - AFP Fact Check;
          - Reuters Fact Check;
          - Snopes;
          - PolitiFact;
          - FactCheck.org;
        - Organizações e Redes Internacionais:
          - International Fact-Checking Network (IFCN);
          - European Fact-Checking Standards Network (EFCSN);
          - Duke Reporters' Lab;
        Informe se o tópico foi encontrado em alguma das agências de fact-checking e qual foi a conclusão dessas agências (verdadeiro, falso, enganoso, etc.).
        Se encontrado, cite a fonte da agência de fact-checking e um breve resumo da sua análise.
        """,
        description="Agente que verifica o que agências de fact-checking dizem a respeito do tópico",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fatos = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    verificacao_fatos = call_agent(verificador_fatos, entrada_do_agente_verificador_fatos)
    return verificacao_fatos


###############################################
# --- Agente 5: Organizador do Resultado da Verificação --- #
################################################
def agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos):
    organizador = Agent(
        name="agente_organizador_resultado",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de Resultados #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é organizar os resultados com base nas análises dos outros agentes sobre o tópico, e determinar a veracidade da informação.
        Apresente um veredicto claro: Verdadeiro, Falso, Enganoso, Insustentável, etc.
        Justifique sua conclusão de forma concisa, utilizando as evidências e os resultados fornecidos pelos outros agentes (agente_buscador, agente_verificador_fontes, agente_verificador_conteudo e agente_verificador_fatos).
        Preste atenção aos lançamentos buscados e inclua os **links das fontes relevantes** no resultado final, se disponíveis.
        Inclua os **links das fontes relevantes formatados em Markdown ([Fonte - Texto do Link](URL do Link))** no resultado final, se disponíveis.
        Liste as fontes mais relevantes (sites de notícias confiáveis, agências de fact-checking) que sustentam sua conclusão.

        Padrão de Resultado:
        - Escrever um resumo da verifição do tópico neste ponto.
        - Parecer: VERDADEIRO, FALSO, ENGANOSO, INSUSTENTÁVEL, etc.
        - Justificativas:
          - Justificativa 1.
          - Justificativa 2.
          - Justificativa 3.
          - Justificativa n.
        - Fontes relevantes:
          - ([Fonte 1 - Texto do Link da Fonte 1](URL do Link da Fonte 1)).
          - ([Fonte 2 - Texto do Link da Fonte 2](URL do Link da Fonte 2)).
          - ([Fonte 3 - Texto do Link da Fonte 3](URL do Link da Fonte 3)).
          - ([Fonte n - Texto do Link da Fonte n](URL do Link da Fonte n)).
        """,
        description="Agente que organiza os resultados da verificação",
        tools=[google_search]
    )

    entrada_do_agente_organizador_resultado = f"Tópico:{topico}\nLançamentos buscados:{lancamentos_buscados}\nVerificação fontes:{verificacao_fontes}\nVerificação conteúdo:{verificacao_conteudo}\nVerificação fatos:{verificacao_fatos}"
    # Executa o agente
    resultado = call_agent(organizador,  entrada_do_agente_organizador_resultado)
    return resultado

In [24]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos pesquisar sobre a veracidade a respeito de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    verificacao_fontes = agente_verificador_fontes(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---\n")
    display(to_markdown(verificacao_fontes))
    print("--------------------------------------------------------------")

    verificacao_conteudo = agente_verificador_conteudo(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---\n")
    display(to_markdown(verificacao_conteudo))
    print("--------------------------------------------------------------")

    verificacao_fatos = agente_verificador_fatos(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---\n")
    display(to_markdown(verificacao_fatos))
    print("--------------------------------------------------------------")

    resultado = agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos)
    print("\n--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: A Terra é plana
Maravilha! Vamos pesquisar sobre a veracidade a respeito de A Terra é plana

--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---



> Compreendo. Minha função é pesquisar no Google notícias recentes e relevantes sobre o tópico "A Terra é plana", priorizando fontes jornalísticas confiáveis. Apresentarei no máximo 5 lançamentos dos últimos 30 dias, com título, resumo e link. Se houver pouca cobertura, indicarei a baixa relevância do tópico.
> 
> 
> A cobertura noticiosa sobre a teoria da Terra Plana parece limitada e esparsa. A maioria dos artigos encontrados são de anos anteriores e focam em refutar a teoria ou analisar o fenômeno do terraplanismo. No entanto, encontrei alguns lançamentos recentes que podem ser relevantes:
> 
> 1.  **Wiz Khalifa acredita que “Terra é plana”, mas usa planeta redondo em álbum | CNN Brasil**
> 
> *   **Resumo:** O rapper Wiz Khalifa declarou em uma entrevista que acredita que a Terra é plana, justificando com suas experiências de viagem. Contudo, a capa de seu novo álbum apresenta uma imagem da Terra redonda.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHVu-1DN4FVucAHurIGuXiD467piW9JDegxe9f8PMO8AA\_boXszyPtcA3JK9QqJ0HS9COMCUrJNSY6l3lGmqE9-jPjdz5kPLG-XOMznyywkwztiCiF9RYqxqyLGDxyednXak\_-ixSTJGMuit\_FHIKvdjGvx9cWZfkOw5K6N4JmXYD\_BHG8Ka30dXcic9\_kdPD1KjHSPBCa1-18hix6iI9sZFoIfsSV2kGOE5KPsFLQ=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHVu-1DN4FVucAHurIGuXiD467piW9JDegxe9f8PMO8AA_boXszyPtcA3JK9QqJ0HS9COMCUrJNSY6l3lGmqE9-jPjdz5kPLG-XOMznyywkwztiCiF9RYqxqyLGDxyednXak_-ixSTJGMuit_FHIKvdjGvx9cWZfkOw5K6N4JmXYD_BHG8Ka30dXcic9_kdPD1KjHSPBCa1-18hix6iI9sZFoIfsSV2kGOE5KPsFLQ=)
> 2.  **Pina irritado com terraplanistas: "\[Era] uma conspiração mundial há anos e este rapaz foi à Noruega e descobriu" - SIC**
> 
> *   **Resumo:** Um membro do programa "Irritações" investiga as teorias da Terra Plana.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHsOytmcqGSeVJLl6HC4PlO0rGsonGUUcaXc8gHtWTZhLc6QAYv2BRXloT\_cC6ky3iLjqUZHEICOxRsV\_wHOQOG0tz3RB-VIuYO2YtR4NOCBzrjy1ImCledpOqtVsN-0yp44-vmwk2NFi96V34mE6asVIxrE2b1ZejssUJ9Mz\_tkVxlKLgc52KEnDZ5xifFeGq1GSj4blY7YxB\_-xWUCTe4O3BTeHA4nmL4TCQ3SJ9pnpB6-qc3C4LuBC2442NGTKqxxvEs5orqRievwIWWoVz4uNh-V5ZHtEAVpuRQmIn88eD8Yk1JY0KrjaSkjj82zoaZ4HwW](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHsOytmcqGSeVJLl6HC4PlO0rGsonGUUcaXc8gHtWTZhLc6QAYv2BRXloT_cC6ky3iLjqUZHEICOxRsV_wHOQOG0tz3RB-VIuYO2YtR4NOCBzrjy1ImCledpOqtVsN-0yp44-vmwk2NFi96V34mE6asVIxrE2b1ZejssUJ9Mz_tkVxlKLgc52KEnDZ5xifFeGq1GSj4blY7YxB_-xWUCTe4O3BTeHA4nmL4TCQ3SJ9pnpB6-qc3C4LuBC2442NGTKqxxvEs5orqRievwIWWoVz4uNh-V5ZHtEAVpuRQmIn88eD8Yk1JY0KrjaSkjj82zoaZ4HwW)
> 3.  **Terraplanista vai à Antártida “provar” sua teoria, mas acaba admitindo que estava errado**
> 
> *   **Resumo:** A expedição de um terraplanista à Antártida para observar o fenômeno do "Sol da meia-noite" o levou a concluir que a Terra é, de fato, esférica, contrariando suas crenças anteriores.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXElNxrHaceU0k-wVvxAHJpDk2n8SWGkPxz6DRFuQOsFCQArh3p8WX8ibU-qljgotAxCc7TD6gR-QgzvV\_QIwdYU0SS5sGayhvEceQMGXx53RzFSnREn7QU9JJOOFuSTMXfJDsAVKL5IF4cwM-Jjy\_4e\_33lqEsitgFQ61bJezYQk9rwye5K3kcjjCfJ4x2Okl3PxS5TlUzIeK2trbvCBaHMmhvjlJo6Qu5VAmVFEuqr5Ko-rg==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXElNxrHaceU0k-wVvxAHJpDk2n8SWGkPxz6DRFuQOsFCQArh3p8WX8ibU-qljgotAxCc7TD6gR-QgzvV_QIwdYU0SS5sGayhvEceQMGXx53RzFSnREn7QU9JJOOFuSTMXfJDsAVKL5IF4cwM-Jjy_4e_33lqEsitgFQ61bJezYQk9rwye5K3kcjjCfJ4x2Okl3PxS5TlUzIeK2trbvCBaHMmhvjlJo6Qu5VAmVFEuqr5Ko-rg==)
> 4.  **O terraplanista que viajou para ver fenômeno solar e 'descobriu' que a Terra é redonda: 'Foi como uma desilusão amorosa' - BBC News Brasil**
> 
> *   **Resumo:** Um Youtuber que antes promovia a teoria da Terra Plana, após uma viagem para observar um fenômeno solar, concluiu que estava errado e agora tenta refutar a teoria em seu canal.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEe-fuzn7dU1eNJMTUELnyfxxhJ8rCZ5iT63uXOVgdQT-qhsnrT4eiv5f7RCpWSZEOVZejYA3\_L4d6uXiMAt\_Ua\_UF293QhEkzgte8P0VofTxcYJ8shpoyeZClUEokKcqEM3QPrygknHfNIU1GNAQ==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEe-fuzn7dU1eNJMTUELnyfxxhJ8rCZ5iT63uXOVgdQT-qhsnrT4eiv5f7RCpWSZEOVZejYA3_L4d6uXiMAt_Ua_UF293QhEkzgte8P0VofTxcYJ8shpoyeZClUEokKcqEM3QPrygknHfNIU1GNAQ==)
> 
> A cobertura limitada e a natureza dos artigos (refutação, análise do fenômeno) podem indicar que o tópico "A Terra é Plana" não é um tema de grande relevância noticiosa no momento.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---



> Ok, seguindo as instruções, vou analisar as fontes apresentadas para verificar sua confiabilidade e corroborar as informações.
> 
> **1. CNN Brasil:**
> 
> *   **Confiabilidade:** A CNN Brasil é um известном veículo de notícias com reputação generally considered confiável.
> *   **Sobre nós:** A CNN Brasil faz parte da CNN, uma rede global de notícias. Sua missão é fornecer notícias imparciais e precisas.
> *   **Corroboração:**
>     *   Outras fontes de notícias relataram a declaração de Wiz Khalifa sobre a Terra ser plana e a ironia de usar uma imagem da Terra redonda na capa do álbum.
>     *   Exemplo: [https://www.rapmais.com/noticias/wiz-khalifa-acredita-que-a-terra-e-plana-mas-usa-planeta-redondo-em-album/](https://www.rapmais.com/noticias/wiz-khalifa-acredita-que-a-terra-e-plana-mas-usa-planeta-redondo-em-album/)
> 
> **2. SIC (Sociedade Independente de Comunicação):**
> 
> *   **Confiabilidade:** A SIC é uma das principais emissoras de televisão de Portugal, com um histórico de jornalismo investigativo e cobertura noticiosa.
> *   **Sobre nós:** A SIC é um canal de televisão privado português que transmite uma variedade de programação, incluindo notícias, entretenimento e esportes.
> *   **Corroboração:**
>     *   Não encontrei outras fontes noticiosas importantes que cobrissem especificamente o segmento do programa "Irritações" sobre terraplanismo. No entanto, a SIC é geralmente considerada uma fonte confiável em Portugal.
> 
> **3. BBC News Brasil:**
> 
> *   **Confiabilidade:** A BBC News Brasil é um известном veículo de notícias com reputação generally considered confiável.
> *   **Sobre nós:** A BBC é uma organização de mídia pública do Reino Unido, conhecida por seu jornalismo imparcial e de alta qualidade. A BBC News Brasil é sua divisão brasileira.
> *   **Corroboração:**
>     *   Vários outros veículos de notícias relataram a história do terraplanista que viajou para a Antártida e mudou de ideia.
>     *   Exemplos:
>         *   [https://www.terra.com.br/noticias/ciencia/terraplanista-vai-a-antartida-provar-sua-teoria-mas-acaba-admitindo-que-estava-errado,5455a113269f28c3808bb65074a30858k605urtb.html](https://www.terra.com.br/noticias/ciencia/terraplanista-vai-a-antartida-provar-sua-teoria-mas-acaba-admitindo-que-estava-errado,5455a113269f28c3808bb65074a30858k605urtb.html)
>         *   [https://revistagalileu.globo.com/Ciencia/noticia/2024/04/terraplanista-que-foi-para-antartida-diz-que-terra-e-redonda-foi-como-uma-desilusao-amorosa.ghtml](https://revistagalileu.globo.com/Ciencia/noticia/2024/04/terraplanista-que-foi-para-antartida-diz-que-terra-e-redonda-foi-como-uma-desilusao-amorosa.ghtml)
> 
> **Conclusão:**
> 
> As fontes apresentadas (CNN Brasil, SIC e BBC News Brasil) são geralmente confiáveis. As informações sobre o terraplanista que viajou para a Antártida e mudou de ideia foram amplamente divulgadas por outros veículos de notícias. A declaração de Wiz Khalifa também foi noticiada por outras fontes.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---



> Para verificar se a afirmação de que "A Terra é plana" já foi alvo de checagem por agências de fact-checking confiáveis, vou realizar uma busca no Google utilizando os nomes das agências mencionadas e a frase em questão.
> 
> Sim, a afirmação de que "A Terra é plana" já foi alvo de checagem por diversas agências de fact-checking. A conclusão é que essa afirmação é falsa.
> 
> **Boatos.org:**
> 
> *   Em 06/05/2025, o Boatos.org publicou um artigo desmentindo a alegação de que "A Bíblia e experimentos comprovam que a Terra é plana". A análise conclui que teorias da conspiração sobre o formato da Terra reaparecem com frequência na internet, tentando atacar o conhecimento científico estabelecido com argumentos pseudorreligiosos ou experiências supostamente empíricas. O artigo também ressalta o consenso científico de que a Terra tem formato de geoide, baseado em séculos de observações, cálculos e imagens de satélites.
> *   Em 31/05/2019, o Boatos.org desmentiu um boato de que o General Heleno teria afirmado, em entrevista ao SBT, que a Terra é plana.
> 
> **UOL Confere:**
> 
> *   O UOL Educação publicou um artigo sobre a crescente popularidade da teoria da Terra Plana, explicando que terraplanistas acreditam em uma conspiração mundial para esconder a verdade sobre o formato da Terra e que a ciência manipula a população para acreditar que ela é esférica. O artigo também apresenta refutações científicas da teoria da Terra Plana e discute o fenômeno da pós-verdade, onde fatos objetivos têm menos influência do que crenças pessoais.
> 
> **Outras Agências e Fontes:**
> 
> *   Outras agências de fact-checking, como a AFP Fact Check, Reuters Fact Check, Snopes, PolitiFact e FactCheck.org, também refutam consistentemente a alegação de que a Terra é plana.
> *   A Agência Pública publicou um artigo em 05/06/2020 mencionando o negacionismo de fatos, como a crença de que a Terra é plana, como um tema relevante no mundo atual.
> 
> As análises das agências de fact-checking e das fontes científicas são unânimes em refutar a teoria da Terra plana, apresentando diversas evidências que comprovam que a Terra é um geoide (aproximadamente esférica).


--------------------------------------------------------------

--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---



> - A teoria de que a Terra é plana é um tema que, apesar de amplamente refutado pela ciência, ainda encontra espaço em discussões e notícias. Recentemente, celebridades como Wiz Khalifa expressaram crenças nesse sentido, enquanto outras notícias relatam a experiência de terraplanistas que, ao confrontarem-se com evidências em expedições, mudaram de opinião. Agências de fact-checking e veículos de notícias confiáveis consistentemente desmentem a teoria da Terra Plana, reiterando o consenso científico de que a Terra é um geoide.
> - Parecer: FALSO
> - Justificativas:
>   - A teoria da Terra Plana é contrária a séculos de evidências científicas, observações, cálculos e imagens de satélites.
>   - Agências de fact-checking e veículos de notícias confiáveis desmentem consistentemente a teoria da Terra Plana.
>   - Relatos de terraplanistas que mudaram de opinião após confrontarem-se com a realidade reforçam a falsidade da teoria.
> - Fontes relevantes:
>   - ([Wiz Khalifa acredita que “Terra é plana”, mas usa planeta redondo em álbum | CNN Brasil](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHVu-1DN4FVucAHurIGuXiD467piW9JDegxe9f8PMO8AA_boXszyPtcA3JK9QqJ0HS9COMCUrJNSY6l3lGmqE9-jPjdz5kPLG-XOMznyywkwztiCiF9RYqxqyLGDxyednXak_-ixSTJGMuit_FHIKvdjGvx9cWZfkOw5K6N4JmXYD_BHG8Ka30dXcic9_kdPD1KjHSPBCa1-18hix6iI9sZFoIfsSV2kGOE5KPsFLQ=)).
>   - ([Pina irritado com terraplanistas: "\[Era] uma conspiração mundial há anos e este rapaz foi à Noruega e descobriu" - SIC](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHsOytmcqGSeVJLl6HC4PlO0rGsonGUUcaXc8gHtWTZhLc6QAYv2BRXloT_cC6ky3iLjqUZHEICOxRsV_wHOQOG0tz3RB-VIuYO2YtR4NOCBzrjy1ImCledpOqtVsN-0yp44-vmwk2NFi96V34mE6asVIxrE2b1ZejssUJ9Mz_tkVxlKLgc52KEnDZ5xifFeGq1GSj4blY7YxB_-xWUCTe4O3BTeHA4nmL4TCQ3SJ9pnpB6-qc3C4LuBC2442NGTKqxxvEs5orqRievwIWWoVz4uNh-V5ZHtEAVpuRQmIn88eD8Yk1JY0KrjaSkjj82zoaZ4HwW)).
>   - ([Terraplanista vai à Antártida “provar” sua teoria, mas acaba admitindo que estava errado](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXElNxrHaceU0k-wVvxAHJpDk2n8SWGkPxz6DRFuQOsFCQArh3p8WX8ibU-qljgotAxCc7TD6gR-QgzvV_QIwdYU0SS5sGayhvEceQMGXx53RzFSnREn7QU9JJOOFuSTMXfJDsAVKL5IF4cwM-Jjy_4e_33lqEsitgFQ61bJezYQk9rwye5K3kcjjCfJ4x2Okl3PxS5TlUzIeK2trbvCBaHMmhvjlJo6Qu5VAmVFEuqr5Ko-rg==)).
>   - ([O terraplanista que viajou para ver fenômeno solar e 'descobriu' que a Terra é redonda: 'Foi como uma desilusão amorosa' - BBC News Brasil](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEe-fuzn7dU1eNJMTUELnyfxxhJ8rCZ5iT63uXOVgdQT-qhsnrT4eiv5f7RCpWSZEOVZejYA3_L4d6uXiMAt_Ua_UF293QhEkzgte8P0VofTxcYJ8shpoyeZClUEokKcqEM3QPrygknHfNIU1GNAQ==)).
>   - ([Boatos.org - A Bíblia e experimentos comprovam que a Terra é plana? Entenda!](https://www.boatos.org/ciencia/biblia-experimentos-comprovam-terra-plana.html)).
>   - ([UOL - Educação - Por que a Terra Plana voltou a ganhar tantos adeptos?](https://educacao.uol.com.br/noticias/2024/01/15/por-que-a-terra-plana-voltou-a-ganhar-tantos-adeptos.htm)).
> 


--------------------------------------------------------------
